# Final code

In [1]:
import pandas as pd

In [33]:
base_df = pd.read_csv(r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Ashraf\Pivot Processing\Output\Users_Lang_Skills.csv")

In [34]:
base_df.columns

Index(['USER_ID', 'LAST_NAME', 'FIRST_NAME', 'ANNEES_XP', 'Airbyte_level',
       'Alteryx Analytics Hub_level', 'Alteryx Designer_level',
       'Amazon AWS CloudFormation_level', 'Amazon AWS Glue_level',
       'Amazon QuickSight_level',
       ...
       'Teradata Vantage_level', 'Terraform_level', 'VBA_level',
       'Vena Close_level',
       'Vena Solutions Budgeting, Planning & Forecasting_level',
       'Workday Adaptive Planning_level', 'Workiva_level',
       'insightsoftware Longview Planning_level', 'English_level',
       'French_level'],
      dtype='object', length=139)

In [35]:
exclude_columns = ['USER_ID','LAST_NAME','FIRST_NAME','ANNEES_XP','English_level','French_level']

In [36]:
# Apply transformation to all columns except those in exclude_columns
base_df.columns = [
    col.replace('_level', '').lower() if col not in exclude_columns else col
    for col in base_df.columns
]

In [37]:
base_df.head()

,USER_ID,LAST_NAME,FIRST_NAME,ANNEES_XP,airbyte,alteryx analytics hub,alteryx designer,amazon aws cloudformation,amazon aws glue,amazon quicksight,...,teradata vantage,terraform,vba,vena close,"vena solutions budgeting, planning & forecasting",workday adaptive planning,workiva,insightsoftware longview planning,English_level,French_level
0,2843838,Ramos,David,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0
1,2479537,Small,Carl,2.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,90.0,100.0
2,2533337,Evans,Carmen,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,100.0
3,2446382,Pittman,Brandi,2.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,100.0
4,2433124,Thomas,Julie,2.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0


In [38]:
selected_job = 'Data Engineer'

In [39]:
import pandas as pd

# Define the base path and job name
base_path = r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Ashraf\Mapping\Output"
job_name = 'data_analyst_scores'

# Construct the full file path safely
file_path = f"{base_path}\\{job_name}.csv"

# Read the CSV file
try:
    data_analyst_df = pd.read_csv(file_path)
    print(f"Successfully loaded data from: {file_path}")
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except Exception as e:
    print(f"Error reading file: {e}")


Successfully loaded data from: C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Ashraf\Mapping\Output\data_analyst_scores.csv


In [40]:
exclude_columns_1 = ['USER_ID']

In [41]:
data_analyst_df.head()

,USER_ID,weight_required_skills,weight_preferred_skills,weight_language,req_SQL_score,req_Microsoft Power BI_score,req_Databricks_score,req_DAX_score,req_Matillion ETL_score,lang_English_score,lang_French_score,pref_Cloudera Data Platform_score
0,2843838,13,1,6,150.0,0.0,0.0,0.0,0.0,300.0,300.0,0.0
1,2479537,13,1,6,0.0,0.0,0.0,0.0,0.0,270.0,300.0,0.0
2,2533337,13,1,6,150.0,0.0,0.0,40.0,0.0,150.0,300.0,0.0
3,2446382,13,1,6,240.0,240.0,0.0,0.0,0.0,180.0,300.0,0.0
4,2433124,13,1,6,0.0,0.0,0.0,0.0,0.0,300.0,300.0,0.0


In [42]:
import pandas as pd

# Assuming your DataFrame is named 'df'
columns = data_analyst_df.columns
always_list = ['USER_ID']
req_list = [col for col in columns if col.startswith('req_')] + ['weight_required_skills']
pref_list = [col for col in columns if col.startswith('pref_')] + ['weight_preferred_skills']
weights_list = ['weight_required_skills', 'weight_preferred_skills','weight_language']
lang_list = [col for col in columns if col.startswith('lang_')] + ['weight_language']

# Normalized Language Score 

In [43]:
# Step 1: Identify all language score columns (starting with 'lang_' and ending with '_score')
lang_score_columns = [col for col in data_analyst_df.columns 
                     if col.startswith('lang_') and col.endswith('_score')]

# Step 2: Calculate normalized language score
data_analyst_df['normalized_lang_score'] = (
    data_analyst_df[lang_score_columns].sum(axis=1) / 
    data_analyst_df['weight_language']
)

# Optional: Round to 2 decimal places
data_analyst_df['normalized_lang_score'] = data_analyst_df['normalized_lang_score'].round(1)


# Normalized Required score

In [44]:
# Step 1: Identify all req_*_score columns
req_score_columns = [col for col in data_analyst_df.columns 
                     if col.startswith('req_') and col.endswith('_score')]

# Step 2: Sum the req_*_scores for each row and divide by weight_required_skills
data_analyst_df['normalized_req_score'] = (
    data_analyst_df[req_score_columns].sum(axis=1) / 
    data_analyst_df['weight_required_skills']
)

# Optional: Round the result to 2 decimal places
data_analyst_df['normalized_req_score'] = data_analyst_df['normalized_req_score'].round(1)

# Normalized preferred score

In [45]:
# Step 1: Identify all pref_*_score columns
pref_score_columns = [col for col in data_analyst_df.columns 
                     if col.startswith('pref_') and col.endswith('_score')]

# Step 2: Sum the pref_*_scores for each row and divide by weight_preferred_skills
data_analyst_df['normalized_pref_score'] = (
    data_analyst_df[pref_score_columns].sum(axis=1) / 
    data_analyst_df['weight_preferred_skills']
)

# Optional: Round the result to 2 decimal places
data_analyst_df['normalized_pref_score'] = data_analyst_df['normalized_pref_score'].round(1)

# Normalized Availability Score

In [46]:
import pandas as pd

df_staffing = pd.read_csv(r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Ashraf\Invert Months Processing\Input\HCK_HEC_STAFFING.csv")  

# Function to invert availability values (0 = available, 100 = unavailable)
def invert_availability_values(staffing_df: pd.DataFrame) -> pd.DataFrame:
    """
    Inverts the availability values in the month columns of the staffing DataFrame.
    Converts 0 to 100 (available) and 100 to 0 (unavailable), and inverts other values proportionally.

    Parameters:
        staffing_df (pd.DataFrame): The original staffing DataFrame with month columns.

    Returns:
        pd.DataFrame: The modified staffing DataFrame with inverted availability values.
    """
    # Define month columns (MONTH_1 to MONTH_12)
    month_cols = [f"MONTH_{i}" for i in range(1, 13)]

    # Invert values: 100 - current value for the month columns
    staffing_inverted = staffing_df.copy()
    staffing_inverted[month_cols] = 100 - staffing_inverted[month_cols]

    return staffing_inverted

# Function to save the inverted staffing DataFrame to CSV
def save_inverted_staffing_to_csv(staffing_inverted_df: pd.DataFrame, file_name: str = "staffing_inverted.csv"):
    """
    Saves the inverted staffing DataFrame to a CSV file.

    Parameters:
        staffing_inverted_df (pd.DataFrame): The modified staffing DataFrame.
        file_name (str): The name of the file to save the CSV as.
    """
    staffing_inverted_df.to_csv(file_name, index=False)
    print(f"Saved inverted staffing data to '{file_name}'.")

def add_availability_score_to_users(score_df: pd.DataFrame, selected_job: str) -> pd.DataFrame:
    import json
    import re
    import os
    os.chdir(r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Datasets\Outputs\Job_description_features")
    # === Step 1: Load job features JSON ===
    file_path = f"{selected_job.replace(' ', '_')}_features.json"
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            features = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"No JSON file found for job '{selected_job}'")

    # === Step 2: Extract duration from nested 'Additional Notes' ===
    notes_dict = features.get("Additional Notes", {})
    duration_text = str(notes_dict.get("Duration", ""))
    match = re.search(r"(\d+)\s*month", duration_text)
    duration = int(match.group(1)) if match else 0

    # === Step 3: Define scoring logic ===
    def check_availability_row(row, duration):
        values = row.tolist()
        max_score = 0
        for i in range(len(values) - duration + 1):
            window = values[i:i + duration]
            if all(v == 100 for v in window):
                return 100, "full block"
            elif 0 not in window:
                avg = sum(window) / duration
                max_score = max(max_score, avg)
        return max_score, "avg block" if max_score > 0 else "no block"

    # === Step 4: Score each user ===
    scores = []
    month_cols = [f"MONTH_{i}" for i in range(1, 13)]

    for user_id in score_df["USER_ID"]:
        staff_row = staffing_inverted[staffing_inverted["USER_ID"] == user_id]

        if staff_row.empty:
            scores.append(0)
            continue

        months = staff_row.iloc[0][month_cols]
        score, reason = check_availability_row(months, duration)
        scores.append(score)

    # === Step 5: Merge scores into final DataFrame ===
    result = score_df.copy()
    result["availability_score"] = scores

    return result

In [47]:
# Assuming HCK_HEC_STAFFING is already loaded
staffing_inverted = invert_availability_values(df_staffing)
score_df_Data_Engineer = add_availability_score_to_users(staffing_inverted, selected_job)

# Normalized Cosine similarity score

In [48]:
import pandas as pd

# Define the file path
file_path = r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Nehal\JSON Embedding and Matching\OUTPUTS\user_similarity_ranking.csv"

# Step 1: Read just the column names to check if 'user_rank' exists
columns_to_read = pd.read_csv(file_path, nrows=0).columns.tolist()

# Step 2: Remove 'user_rank' if it exists
if 'user_rank' in columns_to_read:
    columns_to_read.remove('user_rank')

# Step 3: Read the CSV again, excluding 'user_rank' if present
sim_df = pd.read_csv(file_path, usecols=columns_to_read)

# Convert similarity_score to percentage (0-100)
sim_df['similarity_score'] = sim_df['similarity_score'].apply(lambda x: round(x * 100,1))

# Normalized Experience Score

In [49]:
import json
import os

Users_Lang_Skills = pd.read_csv(r'C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Ashraf\Pivot Processing\Output\Users_Lang_Skills.csv')

def add_extracted_experience_score_to_users(score_df: pd.DataFrame, selected_job: str) -> pd.DataFrame:
    Users_Lang_Skills = pd.read_csv(r'C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Ashraf\Pivot Processing\Output\Users_Lang_Skills.csv')
    """
    Adds an 'experience_score' column to score_df based on the experience required
    for the selected job, loaded from a JSON features file.

    The scoring is binned:
        - ≥ required + 2 years → 100
        - > required + 2 to 4 → 70
        - > required + 4       → 50
        - == required          → 100
        - < required by ≤ 2    → 50
        - < required by 2–4    → 30
        - < required by >4     → 10
        - Missing experience   → 0
    """
    os.chdir(r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Datasets\Outputs\Job_description_features")
    # === Step 1: Load job features JSON ===
    file_path = f"{selected_job.replace(' ', '_')}_features.json"
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            features = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"No JSON file found for job '{selected_job}'")

    required_exp = features.get("Experience Required", 0)

    df = score_df.copy()

    def calculate_score(user_exp):
        if pd.isna(user_exp):
            return 0
        delta = user_exp - required_exp
        if delta >= 2:
            return 100
        elif 0 <= delta < 2:
            return 100
        elif -2 < delta < 0:
            return 50
        elif -4 < delta <= -2:
            return 30
        elif delta <= -4:
            return 10
        else:
            return 0

    # Map score using ANNEES_XP from Users_Lang_Skills
    experience_series = Users_Lang_Skills.set_index("USER_ID")["ANNEES_XP"]
    df = df.set_index("USER_ID")
    df["experience_score"] = experience_series.reindex(df.index).apply(calculate_score).fillna(0)

    return df.reset_index()


In [50]:
# Then add experience scores from JSON
score_df = add_extracted_experience_score_to_users(base_df, selected_job)

# Normalized Unmatched Score

In [51]:
import pandas as pd
# Assuming your DataFrame is named 'df'
columns = data_analyst_df.columns
list1 = [col.replace('req_', '').replace('_score', '').lower() for col in columns if col.startswith('req_')]
list2 = [col.replace('pref_', '').replace('_score', '').lower() for col in columns if col.startswith('pref_')]
weights_list = ['weight_required_skills', 'weight_preferred_skills','weight_language']
lang_list = [col for col in columns if col.startswith('lang_')] + ['weight_language']
exclude_columns_base = ['USER_ID','LAST_NAME','FIRST_NAME','ANNEES_XP','English_level','French_level']
unmatched_skills_cols =  [col for col in base_df.columns if col not in list1 + list2 + lang_list + weights_list + exclude_columns_base]

In [52]:
exclude_columns_1 = ['USER_ID']
unmatched_df = base_df[exclude_columns_1 + unmatched_skills_cols]
# Get all columns except excluded ones
columns_to_average = [col for col in unmatched_df.columns 
                     if col not in exclude_columns_1]

# Calculate average score (sum of remaining columns divided by count)
unmatched_df['unmatched_skills_score'] = unmatched_df[columns_to_average].sum(axis=1) / len(columns_to_average)

# Optional: Round to 2 decimal places
unmatched_df['unmatched_skills_score'] = unmatched_df['unmatched_skills_score'].round(1)

C:\Users\hp\AppData\Local\Temp\ipykernel_20068\3857539597.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched_df['unmatched_skills_score'] = unmatched_df[columns_to_average].sum(axis=1) / len(columns_to_average)
C:\Users\hp\AppData\Local\Temp\ipykernel_20068\3857539597.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched_df['unmatched_skills_score'] = unmatched_df['unmatched_skills_score'].round(1)


In [53]:
# Join availability_score first
data_analyst_df = pd.merge(
    data_analyst_df,
    score_df_Data_Engineer[['USER_ID', 'availability_score']],
    on='USER_ID',
    how='left'
)

# Then join similarity_score
data_analyst_df = pd.merge(
    data_analyst_df,
    sim_df[['USER_ID', 'similarity_score']],
    on='USER_ID',
    how='left'
)

# Finally join unmatched_skills_score
data_analyst_df = pd.merge(
    data_analyst_df,
    unmatched_df[['USER_ID', 'unmatched_skills_score']],
    on='USER_ID',
    how='left'
)

# Finally join experience_score
data_analyst_df = pd.merge(
    data_analyst_df,
    score_df[['USER_ID', 'experience_score']],
    on='USER_ID',
    how='left'
)
data_analyst_df.fillna(0, inplace=True)

# Calculating final Normalized score

In [54]:
def calculate_adaptive_weights(df):
    """Dynamic weights with new base distribution and conditional adjustments"""
    # Base weights (total = 0.65)
    base_weights = {
        'normalized_lang_score': 0.20,    # Language
        'experience_score': 0.25,         # Experience (new higher weight)
        'similarity_score': 0.15,         # Similarity
        'availability_score': 0.05        # Availability
    }
    
    # Skill weights (total = 0.35)
    skill_weights = {
        'normalized_req_score': 0.25,     # Required skills
        'normalized_pref_score': 0.10     # Preferred skills
    }
    
    # Detect available columns
    req_exists = 'normalized_req_score' in df.columns
    pref_exists = any(col.startswith('pref_') and col.endswith('_score') 
                     for col in df.columns)
    
    # Case 1: Both skill types exist
    if req_exists and pref_exists:
        weights = {**base_weights, **skill_weights}
    
    # Case 2: Only required skills exist
    elif req_exists and not pref_exists:
        weights = {**base_weights, 'normalized_req_score': 0.35}
    
    # Case 3: Only preferred skills exist (unlikely)
    elif pref_exists and not req_exists:
        weights = {**base_weights, 'normalized_pref_score': 0.35}
    
    # Case 4: No skill columns - redistribute
    else:
        weights = base_weights.copy()
        # Redistribute skill weight proportionally to base weights
        total_base = sum(base_weights.values())
        for k in weights:
            weights[k] += (0.35 * (base_weights[k]/total_base))
    
    # Final validation
    total = sum(weights.values())
    assert abs(total - 1.0) < 0.001, f"Invalid weight sum: {total}"
    
    return weights

# Calculate weights
adaptive_weights = calculate_adaptive_weights(data_analyst_df)

# Calculate candidate score (with safe column access)
score_components = {
    col: data_analyst_df[col] * weight 
    for col, weight in adaptive_weights.items() 
    if col in data_analyst_df.columns
}

data_analyst_df['Candidate_score'] = (
    sum(score_components.values())
).round(2)

# Show results
# print("Applied weights:", {k: round(v, 3) for k, v in adaptive_weights.items()})
result_cols = ['USER_ID', 'Candidate_score'] + list(adaptive_weights.keys())

In [101]:
print("Applied weights:", {k: round(v, 3) for k, v in adaptive_weights.items()})

Applied weights: {'normalized_lang_score': 0.2, 'experience_score': 0.25, 'similarity_score': 0.15, 'availability_score': 0.05, 'normalized_req_score': 0.25, 'normalized_pref_score': 0.1}


In [55]:
data_analyst_df.sort_values(by='Candidate_score', ascending=False, inplace=True)
data_analyst_df = data_analyst_df[data_analyst_df['availability_score'] != 0]

# Saving the final file with scores

In [56]:
import os
from pathlib import Path

# Save with dynamic filename
output_dir = r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Datasets\Outputs\Final-output"
filename = f"candidates_scores_{selected_job.lower().replace(' ', '_')}.csv"
output_path = Path(output_dir) / filename

data_analyst_df.to_csv(output_path, index=False)
print(f"Saved candidate scores for {selected_job} to:\n{output_path}")

Saved candidate scores for Data Engineer to:
C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Datasets\Outputs\Final-output\candidates_scores_data_engineer.csv


In [58]:
pd.set_option('display.max_columns', None)

In [129]:
data_analyst_df1 = pd.read_csv(r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Datasets\Outputs\Final-output\candidates_scores_scrum_master.csv")

In [130]:
data_analyst_df1[['USER_ID', 'Candidate_score']].head(5)

,USER_ID,Candidate_score
0,2433136,70.36
1,2676328,64.89
2,2579999,61.00
3,2540340,60.22
4,2551856,58.04


In [131]:
data_analyst_df2 = pd.read_csv(r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Datasets\Outputs\Final-output\candidates_scores_data_engineer.csv")

In [132]:
data_analyst_df2[['USER_ID', 'Candidate_score']].head(5)

,USER_ID,Candidate_score
0,2433111,55.84
1,2551856,55.21
2,2445149,54.79
3,2433112,54.75
4,2447746,53.73


In [133]:
data_analyst_df2['Demand_id'] = 'Data Engineer'

In [134]:
data_analyst_df1['Demand_id'] = 'Scrum'

In [153]:
data_analyst_df3 = pd.read_csv(r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Datasets\Outputs\Final-output\candidates_scores_data_engineer.csv")

In [154]:
data_analyst_df3 = data_analyst_df3[['USER_ID','Candidate_score']]

In [155]:
data_analyst_df3.head()

,USER_ID,Candidate_score
0,2433111,55.84
1,2551856,55.21
2,2445149,54.79
3,2433112,54.75
4,2447746,53.73


In [160]:
# Filter for the specific user
user_row = base_df[base_df['USER_ID'] == 2447746]

# Select columns where value is not 0 (for numeric columns)
non_zero_cols = user_row.loc[:, (user_row != 0).any(axis=0)]

# Display the result
non_zero_cols

,USER_ID,LAST_NAME,FIRST_NAME,ANNEES_XP,dax,databricks lakehouse platform,microsoft power bi,sql,snowflake data cloud,English_level,French_level
14,2447746,Payne,Paul,2.5,70.0,60.0,100.0,70.0,80.0,70.0,100.0


In [137]:
data_analyst_df3 = data_analyst_df3[['USER_ID','Candidate_score']]
data_analyst_df3['Demand_id'] = 'Data Analyst'

In [139]:
pd.concat([data_analyst_df1[['Demand_id','USER_ID','Candidate_score']].head(5),data_analyst_df2[['Demand_id','USER_ID','Candidate_score']].head(5),data_analyst_df3[['Demand_id','USER_ID','Candidate_score']].head(5)]).to_csv(r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Datasets\Outputs\Final-output\candidates_scores.csv",index=False)

In [90]:
data_analyst_df.head()

,USER_ID,weight_required_skills,weight_preferred_skills,weight_language,req_R_score,req_Atlassian JIRA Software_score,req_Microsoft Power BI_score,lang_French_score,lang_English_score,normalized_lang_score,normalized_req_score,normalized_pref_score,availability_score,similarity_score,unmatched_skills_score,experience_score,Candidate_score
0,2433136,9,0,5,200.0,200.0,200.0,300.0,200.0,100.0,66.7,0.0,75.0,71.8,6.1,50,70.36
1,2676328,9,0,5,160.0,200.0,180.0,300.0,180.0,96.0,60.0,0.0,50.0,64.6,9.6,50,64.89
2,2579999,9,0,5,80.0,160.0,200.0,300.0,200.0,100.0,48.9,0.0,75.0,50.9,11.8,50,61.00
3,2540340,9,0,5,40.0,100.0,120.0,300.0,140.0,88.0,28.9,0.0,100.0,100.0,4.4,50,60.22
4,2551856,9,0,5,80.0,0.0,200.0,300.0,160.0,92.0,31.1,0.0,75.0,83.4,8.2,50,58.04


NameError: name 'job_data' is not defined

,USER_ID,LAST_NAME,FIRST_NAME,ANNEES_XP,dax,microsoft power bi,microsoft sql server,sql,snowflake data cloud,tableau desktop and online,English_level,French_level
79,2433111,Richards,William,2.5,70.0,80.0,80.0,100.0,90.0,70.0,100.0,100.0
